In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
data = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')

In [3]:
data.head()


,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
data = np.array(data)

np.random.shuffle(data)

m,n = data.shape 


data_dev = data[0:1000].T
#transpose so every coloumn is an example
Y_dev = data_dev[0] # first row
X_dev = data_dev[1:n]


data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]

In [5]:
#initialize parameters

def init_param():
    W1 = np.random.rand(10,784)  #generates random values between 0 and 1
    b1 = np.random.rand(10,1) 
    W2 = np.random.rand(10,10) 
    b2 = np.random.rand(10,1) 
    return W1,b1,W2,b2

#define ReLU
def ReLU(Z):
 return np.maximum(0,Z) 

#maximum goes through each element in Z, if greater than 0, returns Z

def softmax(Z):
    Z_exp = np.exp(Z - np.max(Z, axis=0, keepdims=True))  # Subtract max for numerical stability
    return Z_exp / np.sum(Z_exp, axis=0, keepdims=True)



#define forward propogation
def forward_prop(W1,b1,W2,b2,X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1,A1,Z2,A2


#one hot code func for Y
def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max()+1))
    one_hot_Y[np.arange(Y.size), Y ] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def deriv_ReLU(Z):
    return Z > 0


#define backward propagation
def back_prop(Z1,A1,Z2,A2,W2,X,Y):
    #need to one hot encode Y
    m = Y.size
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)

    dZ1 = W2.T.dot(dZ2) * deriv_ReLU(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    
    db2 = 1 / m * np.sum(dZ2, axis=1, keepdims=True)
    db1 = 1 / m * np.sum(dZ1, axis=1, keepdims=True)

    return dW1, db1,dW2,db2

def update_pamas(W1,b1,W2,b2,dW1,db1,dW2,db2,alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    return W1,b1,W2,b2
    
    

    
    

In [6]:
#np.argmax(A2, 0): Finds the index of the maximum value along the 0th axis (rows).
#This corresponds to the class with the highest predicted probability for each example.

def get_predictions(A2):
    return np.argmax(A2,0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X,Y,iterations,alpha):
    #get our initial values 
    W1,b1,W2,b2 = init_param()
    for i in range(iterations):
        Z1,A1,Z2,A2  = forward_prop(W1,b1,W2,b2,X)
        dW1 , db1, dW2, db2 = back_prop(Z1,A1,Z2,A2, W2,X,Y)
        W1 , b1,W2,b2 =  update_pamas(W1,b1,W2,b2,dW1,db1,dW2,db2,alpha)
        if i % 10 == 0 :
            print("Iteration : ", i )
            print("Accuracy : ", get_accuracy(get_predictions(A2),Y))
    return W1,b1,W2,b2
        

In [7]:
W1,b1,W2,b2 = gradient_descent(X_train, Y_train, 500, 0.10)


Iteration :  0
[0 0 0 ... 0 0 0] [0 1 8 ... 7 5 9]
Accuracy :  0.09853658536585366
Iteration :  10
[2 2 2 ... 2 2 2] [0 1 8 ... 7 5 9]
Accuracy :  0.09946341463414635
Iteration :  20
[2 2 2 ... 2 2 2] [0 1 8 ... 7 5 9]
Accuracy :  0.09946341463414635
Iteration :  30
[2 2 2 ... 2 2 2] [0 1 8 ... 7 5 9]
Accuracy :  0.09946341463414635
Iteration :  40
[2 2 2 ... 2 2 2] [0 1 8 ... 7 5 9]
Accuracy :  0.09946341463414635
Iteration :  50
[2 2 2 ... 2 2 2] [0 1 8 ... 7 5 9]
Accuracy :  0.09946341463414635
Iteration :  60
[2 2 2 ... 2 2 2] [0 1 8 ... 7 5 9]
Accuracy :  0.09946341463414635
Iteration :  70
[2 2 2 ... 2 2 2] [0 1 8 ... 7 5 9]
Accuracy :  0.09946341463414635
Iteration :  80
[2 2 2 ... 2 2 2] [0 1 8 ... 7 5 9]
Accuracy :  0.09946341463414635
Iteration :  90
[2 2 2 ... 2 2 2] [0 1 8 ... 7 5 9]
Accuracy :  0.09946341463414635
Iteration :  100
[2 2 2 ... 2 2 2] [0 1 8 ... 7 5 9]
Accuracy :  0.09946341463414635
Iteration :  110
[1 1 1 ... 1 1 1] [0 1 8 ... 7 5 9]
Accuracy :  0.111804878

In [8]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    return predictions

def test_prediction(index, W1, b1, W2, b2):
    current_image = X_train[:, index, None]
    prediction = make_predictions(X_train[:, index, None], W1, b1, W2, b2)
    label = Y_train[index]
    print("Prediction: ", prediction)
    print("Label: ", label)
    
    current_image = current_image.reshape((28, 28)) * 255
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()